In [1]:
import os
import sys
root = '/home/mark/Documents/code/mdrp'
sys.path.append(root)

import numpy as np
from utils.nlp_mdrp import mdrp_prob
import cyipopt

from utils.convert import convert_instance
from utils.nlp_mdrp import mdrp_prob
from utils.helpers import *

Lets create the preference function $\alpha:[0,1]\rightarrow(0,\infty)$.

For now we will only consider a linear function, making it easily invertible.

In [2]:
max_dph = 100
min_dph = 5
alpha = lambda x : (1/max_dph) + ((1/min_dph)-(1/max_dph))*x 
alpha_inv = lambda x: (x-(1/max_dph))/((1/min_dph)-(1/max_dph))

Toy Setting:

- We want somethign easy to visualize but still meaningful (will use ternary plot)
- Two order locations: close + far
- Three delivery modes: drones, cars, robots
- Full formulation for latency, costs, prices... 

**ASSUMPTIONS**
- The total demand of the system is approx. equal to what the system can support under capacity constraints  
- The demand of the far locations is DOUBLE the demand of the close locations
- Each delivery mode can satisfy 1/3 of the total demand (hence all of the demand for the close location)

This can be solved with three unknowns that sum to a constant, and represented by a ternary plot:
- the variables are what portion (0-1) of the total available drones/cars/robots are used for the close location.
- the remaining portion must be used for the far location to satisfy the demand. 


Working backwards, recall that we have capacity constraints $\rho_j$ for each delivery mode. Hence the total order load that each mode can support is:

$$\rho_j N_j \mu_j,$$

where $\mu_j$ is the order service rate (orders/hour) and is computer before hand, and $N_j$ is the number of avaible couriers of that type. We can use this to formulate the problem accordingly. 


In [3]:
# HELPER FUNCTIONS FOR NOTEBOOK #
def get_mu(s_time,t_time,beta,N,k):
    '''
    computers average service rate as the expected value for the given
    problem isntance
    '''
    expected_lat = (s_time+t_time+(k/(1+beta*N*(0.1)))).mean(axis=0)
    print(expected_lat)
    return 1/expected_lat

In [4]:
# Parameters for toy problem (INELASTIC)
# Units and rates are in hours
n_orders = 2
n_modes = 3
prob_setup = {}

# same service time
prob_setup['s_time'] = np.ones((n_orders,n_modes))*(5/60) 
# far vs close
prob_setup['t_time'] = np.array([[10,20,30],[5,10,15]])/60
# same cost 
prob_setup['cost'] = np.ones((n_orders,n_modes))*(5) 
# more robots are located near the center
prob_setup['beta'] = np.array([[1,0.5,0.2],[1,0.5,1]]) 
# Set N so that setting makes sense
prob_setup['N'] = np.array([6.905,10,12.64])
# time radius considered for beta
prob_setup['k'] = np.ones(n_modes)*10/60
# compute mu which depends on N as well
prob_setup['mu'] = get_mu(prob_setup['s_time'],
                          prob_setup['t_time'],
                          prob_setup['beta'],
                          prob_setup['N'],
                          prob_setup['k'])
# check that formulation is valid and all modes have same effective capacity
print('All should be the same: \n',prob_setup['mu']*prob_setup['N'])
prob_setup['max_rho'] = 0.9
prob_setup['max_cost'] = 100000 # irrelevant 
prob_setup['demand'] = 0.999

prob_setup['scale'] = True

[0.30692349 0.44444444 0.56165901]
All should be the same: 
 [22.49746316 22.5        22.50475783]


In [5]:
mdrp = mdrp_prob(prob_setup)
lb, ub, cl, cu = mdrp.get_bounds()
x0 = mdrp.guess_init()

nlp = cyipopt.Problem(
    n=len(x0),
    m=len(cl),
    problem_obj=mdrp,
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu
    )

# nlp.addOption('derivative_test', 'second-order')
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('tol', 1e-4)

x, info = nlp.solve(x0)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:       24
Number of nonzeros in Lagrangian Hessian.............:       21

Objective value at iteration #0 is - 0.336519
Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints..........

In [6]:
eval_setup = {}
eval_setup['max_dph'] = 100
eval_setup['min_dph'] = 5
eval_setup['min_price'] = 10
eval_setup['mode_names'] = ['drones','cars','droids']
eval_setup['speeds'] = [2,1,0.3]

print('Evaluation')
print('Mean delivery rate mu: ',(1/mdrp.mu)*60)
eval_sol(x,mdrp,eval_setup)

Evaluation
Mean delivery rate mu:  [18.41540964 26.66666667 33.69954059]
Problem Evlatuation
--------------------------------------------------------
Orders Demanded (order/hour): 60.69
Maximum Cost ($/hour): 100000.00
--------------------------------------------------------
Mode 		|drones	|cars	|droids	|total	|
----------------|-------|-------|-------|-------|
Orders (%)	|0.33	|0.33	|0.33	|1.00	|
Op. Cost ($/hr)	|101.25	|101.25	|100.95	|303.45	|
Utilization (%)	|0.90	|0.90	|0.90	|0.90	|
D Time(min.)	|15.14	|20.18	|20.08	|18.47	|
Order Price	|12.08	|10.98	|10.00	|11.02	|
Distance	|6.40	|4.80	|1.44	|4.21	|
Total Money from Orders:  668.7197406008974


In [17]:
print(np.round(x.reshape(n_orders,n_modes)/(prob_setup['N']*prob_setup['mu']*prob_setup['max_rho']),1))
print(x)

[[ 1.   0.5 -0. ]
 [-0.   0.5  1. ]]
[ 2.02477171e+01  1.00979064e+01 -3.96972446e-10 -2.95900771e-09
  1.01520938e+01  2.01935296e+01]


In [8]:
# SETUP MATPLOTLIB #
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
# colors used
ORANGE = '#FF9132'
RED   = '#B02105'
PINK = '#E600BF'
TEAL = '#0598B0'
GREEN = '#008F00'
PURPLE = '#8A2BE2'
GRAY = '#969696'
FIG_WIDTH = 3
FIG_HEIGHT = 3.5
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 8,
    "axes.titlesize": 10,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "lines.linewidth": 2
})

fig_path = os.path.join(root,'figures/')



In [9]:
# fig_name = 'beta_1'

# fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
# heights = [0.5,0.5]
# # 3 SUBPLOTS
# widths = [1]
# spec = fig.add_gridspec(ncols=len(widths), 
#                         nrows=len(heights), 
#                         width_ratios=widths,
#                         height_ratios=heights,
#                         left=0.15,
#                         right=0.95,
#                         top=0.92,
#                         bottom=0.1)

# spec.update(wspace=0.15, hspace=0.14)


# axs = [fig.add_subplot(spec[i,0]) for i in range(len(heights))]
# # axs0 will deal with beta0

# # hide other axes
# # axs[1].set_yticklabels([])
# # axs[0] is without retraining
# axs[0].set_title('MNIST',pad=3,loc='right')
# axs[1].set_title('CIFAR',pad=3,loc='right')

# # plot with non robust accuracy shown
# for key in data_keys:
#     data_MNIST = pickle.load(open(os.path.join(root,data_path_b1_MNIST[key]),'rb'))
#     if key not in ['f_0_0']:
#         data_MNIST = smooth(data_MNIST,box_pts)
#     axs[0].plot(k_values,
#                 data_MNIST,
#                 color=data_colors[key],
#                 label=data_labels[key],
#                 linestyle=data_style[key],
#                 zorder=data_zorder[key])

#     if key not in ['abs']:
#         data_CIFAR = pickle.load(open(os.path.join(root,data_path_b1_CIFAR[key]),'rb'))
#         if key not in ['f_0_0']:
#             data_CIFAR = smooth(data_CIFAR,box_pts)
#         axs[1].plot(k_values,
#                 data_CIFAR,
#                 color=data_colors[key],
#                 label=data_labels[key],
#                 linestyle=data_style[key],
#                 zorder=data_zorder[key])
#     # if key not in ['clean_k0']:
#     #     data_beta_1 = smooth(data_beta_1,box_pts)
#     # if key not in ['adv_k0' ,'clean_k0','abs']:
#     #     data_beta_100 = smooth(data_beta_100,box_pts)
#     # print(key)
#     # print(data_beta_1[0])
#     # print(data_beta_100[0])



# axs[0].set_xlim(0,100)
# axs[0].set_ylim(0,100)
# axs[1].set_xlim(0,100)
# axs[1].set_ylim(0,100)

# axs[0].grid()
# axs[1].grid()
# # axs[0].set_xlabel('$\ell_0$--budget/truncation $(k=k_{adv})$\n(a)',labelpad=0)
# # axs[1].set_xlabel('$\ell_0$--budget/truncation $(k=k_{adv})$\n(a)',labelpad=0)
# axs[1].set_xlabel('$\ell_0$--budget',labelpad=0)
# axs[0].set_ylabel('robust accuracy (\%)',labelpad=0)
# axs[1].set_ylabel('robust accuracy (\%)',labelpad=0)
# axs[0].set_xticks([0,12,25,50,75,100])
# # axs[0].set_xticklabels([0,12,25,50,75,100])
# axs[0].set_xticklabels([])
# axs[1].set_xticks([0,12,25,50,75,100])
# axs[1].set_xticklabels([0,12,25,50,75,100])
# axs[1].legend(loc='upper right')
# axs[0].legend(loc='upper right')

# fig.suptitle('$(\\ell_0 + \\ell_\\infty)$--adversary')
# # plt.show()
# plt.savefig(fig_path+fig_name+'.eps')
# plt.savefig(fig_path+fig_name+'.pdf')
